In [ ]:
pip install langgraph langchain langchain-openai langchain_community langchain_fireworks

In [ ]:
import os
import google.generativeai as genai
from google.colab import userdata
api_key=userdata.get('OPENAI_API_KEY')

In [ ]:
import pandas as pd
entry=pd.read_csv("/content/simulated_journal (1).csv")


In [ ]:
entry

In [ ]:
from langchain.agents import tool
from langgraph.prebuilt import ToolNode


@tool
def get_distortion_list():
    """Checklist of cognitive distortions."""
    checklist = {
        0: ("No distortion were detected"),
        1: ("All-or-Nothing Thinking", "Viewing situations in absolute terms, as either all good or all bad. Example: 'I missed my work out today.My whole fitness regimen is ruined'"),
        2: ("Overgeneralization", "Drawing broad conclusions based on limited information. Example: 'One nurse was rude to me, so all medical staff must be rude.'"),
        3: ("Mental Filter", "Focusing solely on negative details and ignoring positive aspects. Example: 'I got five compliments, but one person criticized me, so my work is bad.'"),
        4: ("Should Statements", "Using rigid 'should' or 'must' statements about yourself or others. Example: 'I should always be happy and productive.'"),
        5: ("Labeling", "Reducing yourself or others to a single negative characteristic. Example: 'I failed that task, so I’m a loser.'"),
        6: ("Personalization", "Blaming yourself for situations outside your control or assigning blame to others unjustly. Example: 'It rained on our picnic because I didn’t plan properly.'"),
        7: ("Magnification", "Exaggerating the importance of mistakes or negative events. Example: 'If I don’t pass this test, I would never be successful in my career.'"),
        8: ("Emotional Reasoning", "Believing that because you feel a certain way, it must be true. Example: 'I feel like a failure, so I must be one.'"),
        9: ("Mind Reading", "Assuming you know what others are thinking or why they’re acting a certain way. Example: 'They didn’t text back, so they must dislike me.'"),
        10: ("Fortune-Telling", "Expecting things to go badly without sufficient evidence. Example: 'I was afraid of job interviews, so I decided to start my own business.'")
    }
    return checklist
    tool=get_distortion_list(tool_entry=None)
    tool_node = ToolNode(tool)



In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_fireworks import ChatFireworks

Detector_prompt = ChatPromptTemplate.from_messages([
    (f"""
    You are Watson, an AI assistant focused on detecting cognitive distortions.
    Please carefully analyze the entry and output the dominant cognitive distortion present in the entry. The list of distortions to scan for is available in the {tool}.
    Only choose distortions available in the tool. It is very important that the detected distortion is one of the below 10 distortions,
    1.All-or-nothing thinking
    2.Overgeneralization
    3.Mental filter
    4.Should statements
    5.Labeling
    6.Personalization
    7.Magnification
    8.Emotional reasoning
    9.Mind reading
    10.Fortune-telling
    If the evaluator provides critique, respond with a revised version of your assessment.

    For example:

    Entry: I messed up my presentation at work last month. Since then, I'm scared of group meetings. What if I freeze again?

    Objectively Observe (What does the entry reflect about the state of the person?): The person is scared of messing things up.
    Think: Is their state of mind rational? Are they coping with their current situation? If not, what distortion is affecting their ability to cope?
    Detected Distortion: If they are not coping, identify the distortion that is preventing them. Here, it is 2 (Overgeneralization).
    Reason: Based on one event, they worry that the same mistake will be true for all future instances of that event.

    Output format should be as follows:
    situation: [Brief summary of the person's situation]
    coping: [yes/no, depending on whether the person is coping]
    detected distortion: [only name of the distortion, or 'none' if coping is 'yes',should be only from the checklist]
    reason: [Explanation of why this distortion was chosen]

    Example Output:
    situation: scared of messing up presentation
    coping: no, the person is not coping well with the situation.
    detected distortion: Overgeneralization
    reason: Based on one event, they worry that the same mistake will be true for all instances of that event.

    """),
    MessagesPlaceholder(variable_name="messages"),
])







In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

evaluator_prompt = ChatPromptTemplate.from_messages([
    (f"""You are an Evaluator focused on detecting cognitive distortions and reviewing Detectors results.
    Your task is to independently analyze the entry, determine the cognitive distortion, and then compare your result with Watson’s. If there's a discrepancy, explore alternative distortions and give feedback.

    Instructions:
    1. Independently analyze the entry to identify the dominant cognitive distortion. Use the list of distortions available in the {tool}.
    Only choose distortions from the tool.

    2. Critique and Revise assessment:
       - Compare your result with the detector’s detected distortion.
       - see if the reason is sufficient.
       - Is there an alternate distortion from the *checklist* which predicts it the best?
       - If you found an alternate distortion, rise the feedback to the detector, giving a reason why the alternate distortion is best
       - If not, then out put your assessment

    3. Its very important you mention with a "yes,matched!" if there is a match in assessment between yours and detectors response. if not mention"no match".


    Output format:
    Evaluator detected distortion:
    Detector detected distortion:
    feedback:
    match:

    Example:

    Entry: "I failed one test, so I’ll probably fail all my exams."

    Evaluator  detected distortion:Overgeneralization
    Detector detected distortion: Personalization
    Alternative distortion: Overgeneralization

    feedback: "Could you revise your answer? The person is assuming that one failure means future failures, which is overgeneralization rather than personalization."
    match: no match

    Entry: "My friend didn’t text me back. They must hate me."

    Evaluator  detected distortion: Mind Reading
     Detector detected distortion: Fortune-telling
    Alternative distortion: Mind Reading

    feedback: "Could you revisit the result? The person is assuming what their friend feels without evidence, which fits better with mind reading than fortune-telling."
    match: no match

    Entry: "I made a mistake at work. Now everyone thinks I’m incompetent."

    Evaluator detected distortion: Magnification
    Detector’s detected distortion: Magnification

    feedback: None
    match: yes,matched!
    """),
    MessagesPlaceholder(variable_name="messages")


])







In [ ]:
Insight_provider_prompt = ChatPromptTemplate.from_messages([
    ("""
     Insight Provider Prompt:

"You are an empathetic insight provider designed to help users recognize and reframe cognitive distortions. Follow this three-step process to guide the user toward a healthier perspective:"

Acknowledge: Identify and name the cognitive distortion detected (e.g., all-or-nothing thinking, personalization, etc.).
Counter reason: Challenge the distortion using the relevant counter-label and reasoning theme (see the list below).
Final output: Deliver a friendly, conversational response that blends the counter-reason with
encouragement and empathy and offer insights on their problem that they could be not paying attention to. Add an interactive element to it

Counter Reasoning Themes:

All-or-nothing thinking: Nudge to see Middle Grounds..
Overgeneralization: One experience doesn’t dictate future outcomes. Each situation is unique.see the benefit of the doubt to remember the evidence to support or refute
Distress causing assumption.
Mental filter: Acknowledge both positive and negative aspects to gain a balanced perspective.
Should statements: Replace rigid "should" rules with flexible, self-compassionate language.
Labeling: People are more than their mistakes — focus on behaviors, not identity.
Personalization: You are not responsible for everything. Other people’s emotions are influenced by many factors.
Magnification: Mistakes are human. Ask, “Will this matter long-term?” to gain perspective.
Emotional reasoning: Feelings are valid but not always factual. Use logic to check your emotional conclusions.
Mind reading: You can’t know what others are thinking without clear communication.
Fortune-telling: The future is uncertain. Focus on what you can control in the present.

Example Response Format:

Entry: I did bad at the test today despite of all the effort. May be i'm not meant to be here. Others seem smarter than me. sometimes i feel like just quiting.

detector agent= distortion detected:Nudge to see Middle Grounds.
All-or-nothing thinking
output
acknowledge: All-or-nothing thinking
Counter reason:Nudge to see Middle Grounds.
Final output: Hey! Do not let this one event define your decisions hence forth. Don't you think this is an oppurtunity for learning. Why not relax now and
later reflect on why you did not do the test well. This can be a great chance to strike a conversation with professor or senior. They might like you for
your dillegence and willingness to improve. you got this. I'm here to help you at any time. what do you think?

    """),MessagesPlaceholder(variable_name="messages")
])





In [ ]:
result_df=[]

for index, row in entry.iterrows():
  from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
  from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
  from langchain_fireworks import ChatFireworks
  from typing import Annotated, List, Sequence
  from langgraph.graph import END, StateGraph, START
  from langgraph.graph.message import add_messages
  from langgraph.checkpoint.memory import MemorySaver
  from typing_extensions import TypedDict
  import re


  llm = ChatOpenAI(api_key=api_key)

  detect = Detector_prompt | llm

  request = HumanMessage(content=row['entry'])

  detection = ""



  evaluate = evaluator_prompt | llm



  evaluation_result = ""

  for chunk in evaluate.stream({"messages": [request, HumanMessage(content=detection)]}):
      print(chunk.content, end="")
      evaluation_result += chunk.content


  for chunk in detect.stream({"messages": [request]}):
    print(chunk.content, end="")
  detection += chunk.content

  insight = Insight_provider_prompt | llm



  insight_result = ""



  for chunk in insight.stream({"messages": [request, HumanMessage(content=detection)]}):
      print(chunk.content, end="")
      evaluation_result += chunk.content


  class State(TypedDict):
      messages: Annotated[list, add_messages]


  async def detection_node(state: State) -> State:
      return {"messages": [await detect.ainvoke(state["messages"])]}


  async def evaluator_node(state: State) -> State:
      res = await evaluate.ainvoke(state["messages"])
      return {"messages": [AIMessage(content=res.content)]}


  async def insight_provider_node(state: State) -> State:
      return {"messages": [await insight.ainvoke(state["messages"])]}


  builder = StateGraph(State)
  builder.add_node("Detector", detection_node)
  builder.add_node("Evaluator", evaluator_node)
  builder.add_edge(START, "Detector")
  builder.add_node("Insight provider",insight_provider_node)
  builder.add_edge("Evaluator","Insight provider")


  def should_continue(state: State):
      last_message = state["messages"][-1].content.lower()
      print(last_message)
      if re.search(r"\bno match\b", last_message) and len(state["messages"])< 6 :
            print("Evaluator detected 'no match' — looping back to Detector.")
            return "Detector"

      if re.search(r"\bno match\b", last_message) and len(state["messages"])> 6 :
            print("Reached message limit. Ending interaction.")
            return END


      if re.search(r"\byes,matched!\b",last_message):
            print("Assessments Matched. Ending interaction.")
            return "Insight provider"



      return "Insight provider"




  builder.add_edge("Detector", "Evaluator", )
  builder.add_conditional_edges("Evaluator", should_continue)
  builder.add_edge("Insight provider",END)
  memory = MemorySaver()
  graph = builder.compile(checkpointer=memory)

  config = {"configurable": {"thread_id": "1"}}

  async for event in graph.astream(
      {
          "messages": [
              HumanMessage(
                  content=row['entry']
              )
          ],
      },
      config,
  ):
      print(event)
      print("---")

  state = graph.get_state(config)

  entry= state.values["messages"][0].content
  detector_agent= state.values["messages"][1].content
  evaluator_agent=state.values["messages"][2].content
  detected_distortion=re.findall(r'detected distortion:\s*([^\n]+)(?=\n)',detector_agent)[0]
  insight_provider= state.values["messages"][-1].content


  result_df.append({
      "Patient Entry": [entry],
      "detected_distortion":[detected_distortion],
      "Insight Provider": [insight_provider],
      "Detector": [detector_agent],
      "Evaluator": [evaluator_agent],


  })



In [ ]:
ChatPromptTemplate.from_messages(state.values["messages"]).pretty_print()

In [ ]:
result=pd.DataFrame(result_df)
result.to_csv("result_journal_insightbox.csv")

In [ ]:
result

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass